# Объединение локаций из Instagram и OSM

In [77]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt

In [94]:
df_location_osm = pd.read_csv("../csv/locations/locations_geo.csv")
df_location_inst = pd.read_csv("../csv/locations/locations_inst_geo.csv")

In [95]:
df_location_osm

,id,info,info_en,instagram_location,lon,lat
0,node/1269595927,"скульптура Материал, дерево Старик произведени...","sculpture material, tree old man work of art",[],30.268740,59.978465
1,node/2181181342,Музей художественного стекла,Museum of Art Glass,[282917579],30.268557,59.980350
2,node/3761709953,статуя Лев произведение искусства Исторический,statue lion work of art historical,[],30.269520,59.979386
3,node/3761709956,Летнее кафе Удобство,Summer cafe convenience,[],30.269979,59.977516
4,node/6742144166,"информация, карта размер карты, сайт","Information, card size, site",[],30.269894,59.977468
5,node/6750344125,"естественный, дерево","Natural, tree",[],30.271168,59.979579
6,node/6905536550,статуя произведение искусства,Statue work of art,[],30.269010,59.979954
7,node/300951745,"Западная стрелка Елагина острова естественный,...","Western arrow Yelagin is a natural island, rock",[],30.236411,59.980255
8,node/1157617458,"водный путь, запорные ворота","waterway, shut off gates",[],30.271302,59.981290
9,node/2022962538,"На Елагином Удобство, кафе Кухня, итальянская","Yelagin is convenience, cafe kitchen, Italian",[],30.265328,59.980026


In [96]:
df_location_inst["similar"] = False

In [97]:
df_location_inst.drop_duplicates(["info_en"], inplace=True)

In [98]:
vectorizer = TfidfVectorizer()

for i, info_loc in enumerate(df_location_osm["info_en"].to_list()):
    print(i, info_loc)
    for loc_inst in df_location_inst.to_records():
        ind = loc_inst[0]
        info_loc_text = loc_inst[5]
        X = vectorizer.fit_transform([info_loc, info_loc_text])
        arr = X.toarray()
        sim = cosine_similarity(arr)[0][1]
        if sim > 0.4:
            print("\t", sim, info_loc_text)
            df_location_inst.at[ind, 'similar'] = True


0 sculpture material, tree old man work of art
1 Museum of Art Glass
	 1.0 Museum of Art Glass
2 statue lion work of art historical
3 Summer cafe convenience
4 Information, card size, site
5 Natural, tree
6 Statue work of art
7 Western arrow Yelagin is a natural island, rock
8 waterway, shut off gates
9 Yelagin is convenience, cafe kitchen, Italian
10 Medical center convenience, doctors Emergency Health situation, doctor
11 First Cheburery convenience, fast food
12 "Historical Memorial Bust"
13 Convenience, bicycle parking bicycle parking, wall loops
14 playground
15 Shop, ticket
16 Barrier, entrance
17 Kitchen building museum
18 Kirov Central Park
	 1.0000000000000002 Kirov Central Park
19 Middle Nevka
20 Yelagin island
	 0.5797386715376658 Tennis club "Yelagin Island"
21 The stable building of the museum
22 Eastern arrow
23 Pavilion under the flag
24 Musical pavilion
25 Nature Monument "Yelagin Island" Natural Reserve
26 Green beach
27 Yelaginsky ponds
28 Granite marina
	 0.579738671

In [101]:
df_location_inst.at[23, 'similar'] = True

In [103]:
df_location_inst[~df_location_inst["similar"]]

,loc_id,lon,lat,info,info_en,similar
8,768634267,30.243987,59.980423,Norway Park Elagin,Norva Park Elagin,False
13,177634426498771,30.254245,59.981665,Набережная ЦПКиО,embankment,False
14,569761120142629,30.259785,59.976918,parkrun Елагин остров,Parkrun,False
17,364704557468064,30.266562,59.980119,Petshop Days,Petshop Days,False
19,564635850627512,30.268188,59.979479,Усадьба Джаз,Manor Jazz,False
20,269241190453949,30.258695,59.976971,Фестиваль Тюльпанов,Tulip festival,False
21,173249823604802,30.257181,59.979918,Фестиваль Уличных Театров,Festival of street theaters,False


In [106]:
df_location_inst["instagram_location"] = df_location_inst["loc_id"].map(lambda x: [x])

In [104]:
df_location_osm

,id,info,info_en,instagram_location,lon,lat
0,node/1269595927,"скульптура Материал, дерево Старик произведени...","sculpture material, tree old man work of art",[],30.268740,59.978465
1,node/2181181342,Музей художественного стекла,Museum of Art Glass,[282917579],30.268557,59.980350
2,node/3761709953,статуя Лев произведение искусства Исторический,statue lion work of art historical,[],30.269520,59.979386
3,node/3761709956,Летнее кафе Удобство,Summer cafe convenience,[],30.269979,59.977516
4,node/6742144166,"информация, карта размер карты, сайт","Information, card size, site",[],30.269894,59.977468
5,node/6750344125,"естественный, дерево","Natural, tree",[],30.271168,59.979579
6,node/6905536550,статуя произведение искусства,Statue work of art,[],30.269010,59.979954
7,node/300951745,"Западная стрелка Елагина острова естественный,...","Western arrow Yelagin is a natural island, rock",[],30.236411,59.980255
8,node/1157617458,"водный путь, запорные ворота","waterway, shut off gates",[],30.271302,59.981290
9,node/2022962538,"На Елагином Удобство, кафе Кухня, итальянская","Yelagin is convenience, cafe kitchen, Italian",[],30.265328,59.980026


In [108]:
df_full_loc = pd.concat([df_location_osm, df_location_inst[~df_location_inst["similar"]].rename({"loc_id": "id"}, axis=1)[["id", "lat", "lon", "info", "info_en", "instagram_location"]]])

In [109]:
df_full_loc

,id,info,info_en,instagram_location,lon,lat
0,node/1269595927,"скульптура Материал, дерево Старик произведени...","sculpture material, tree old man work of art",[],30.268740,59.978465
1,node/2181181342,Музей художественного стекла,Museum of Art Glass,[282917579],30.268557,59.980350
2,node/3761709953,статуя Лев произведение искусства Исторический,statue lion work of art historical,[],30.269520,59.979386
3,node/3761709956,Летнее кафе Удобство,Summer cafe convenience,[],30.269979,59.977516
4,node/6742144166,"информация, карта размер карты, сайт","Information, card size, site",[],30.269894,59.977468
...,...,...,...,...,...,...
14,569761120142629,parkrun Елагин остров,Parkrun,[569761120142629],30.259785,59.976918
17,364704557468064,Petshop Days,Petshop Days,[364704557468064],30.266562,59.980119
19,564635850627512,Усадьба Джаз,Manor Jazz,[564635850627512],30.268188,59.979479
20,269241190453949,Фестиваль Тюльпанов,Tulip festival,[269241190453949],30.258695,59.976971


In [110]:
df_full_loc.to_csv("../csv/locations/locations_full.csv", index=False)